In [63]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, binary_crossentropy
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import categorical_crossentropy, sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os

In [3]:
os.listdir('/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/valid/')

['cats', 'dogs']

In [156]:
train_path = '/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/train/'
valid_path = '/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/valid/'
test_path = '/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/test/'

In [3]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), 
                                                         classes=['cats', 'dogs'], batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), 
                                                         classes=['cats', 'dogs'], batch_size=10)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), 
                                                         classes=['cats', 'dogs'], batch_size=10)

In [4]:
# Plot images with labels

def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims=np.array(ims).astype(np.uint8)
        if (ims.shape[-1]!=3):
            ims=ims.transpose((0,2,3,1))
        f=plt.figure(figsize=figsize)
        cols=len(ims)//rows if len(ims)%2==0 else len(ims)//rows +1
        for i in range(len(ims)):
            sp=f.add_subplot(rows,cols,i+1)
            sp.axis('Off')
            if titles is not None:
                sp.set_title(titles[i], fontsize=16)
            plt.imshow(ims[i], interpolation=None if interp else 'none')

In [5]:
imgs, labels = next(train_batches)

In [6]:
plots(imgs, titles=labels)

In [7]:
# Build a simple CNN model using a single Conv2D layer

model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)),
    Flatten(),
    Dense(2, activation='softmax'),
])

In [8]:
# Compile the model

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Fit the data to model using fit_generator called on image generator object

model.fit(train_batches, steps_per_epoch=6, validation_data=valid_batches, 
                    validation_steps=2, epochs=5, verbose=2)

## The simple CNN model performed very poorly in fitting the data, the loss is high and accuracy very poor for training and validation stages

In [10]:
# Predict the test batches

test_imgs, test_labels = next(test_batches)

In [11]:
plots(test_imgs, titles=test_labels)

In [12]:
test_labels = test_labels[:,0]
test_labels

In [13]:
preds = model.predict(test_batches, steps=1, verbose=0)

In [14]:
preds

In [15]:
preds = np.round(preds)

In [16]:
preds[:,0]

# The predictions show that all are cats, which is a very wrong prediction from the simple model

In [17]:
len(test_labels), len(preds[:,0])

In [18]:
type(test_labels[0]), type(preds[:,0][0])

In [19]:
confusion_matrix(test_labels, preds[:,0])

In [20]:
ConfusionMatrixDisplay.from_predictions(test_labels, preds[:,0], labels=[0, 1], display_labels=['Dog', 'Cat'])

## So we see that the model has very high false positive and false negative

## Now we will directly import a trained model and then fine tune it for cats and dogs

In [21]:
# Import the VGG16 pretrained model from keras library

vgg16_model = keras.applications.vgg16.VGG16()

What the hell just happened!! On my laptop this model took more than an hour to download. This is crazy.

In [22]:
# Check summary of the model

vgg16_model.summary()

In [23]:
# We have been creating models of type Sequential

type(model)

In [24]:
# However, the downloaded model is of type functional

type(vgg16_model)

In [25]:
# Copy the functional model layers into a Sequential model

model2 = Sequential()
for layer in vgg16_model.layers:
    model2.add(layer)

In [26]:
# Summary of the new model will show that it is exactly the same

model2.summary()

In [28]:
# And the type of the model is Sequential

type(model2)

In [33]:
# Remove the last Dense output layer because that is classifying one of 1000 classes

model2.pop()

In [34]:
# Check to see that the last output layer is gone

model2.summary()

In [35]:
# Freeze all the layers in the model, these will not be trained in the future training steps

for layer in model2.layers:
    layer.trainable = False

In [36]:
# Now add a last output layer to classify only 2 categories for our cats and dogs

model2.add(Dense(2, activation='softmax'))

In [38]:
# Check the summary of the model

model2.summary()

In [39]:
# Compile the new model

model2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', 
               metrics=['accuracy'])

In [41]:
# Fit the model on the training dataset and check with the validation set

model2.fit(train_batches, steps_per_epoch=6, validation_data=valid_batches, 
                    validation_steps=2, epochs=20, verbose=2)

In [56]:
# Predict using fine tuned VGG16 model

test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

In [57]:
test_labels = test_labels[:,0]
test_labels

In [58]:
preds = model2.predict(test_batches, steps=1, verbose=0)

In [59]:
preds

In [60]:
preds = np.round(preds)

In [61]:
preds[:,0]

In [62]:
ConfusionMatrixDisplay.from_predictions(test_labels, preds[:,0], labels=[0, 1], 
                                        display_labels=['Dog', 'Cat'])

In [64]:
accuracy_score(test_labels, preds[:,0])

# Very good accuracy! I just love this pre trained VGG16 model. Hats off to the creators.